In [1]:
# system packages
import numpy as np
import pandas as pd
import math

#Plotting
import matplotlib.pyplot as mtplt
import matplotlib.gridspec as gridspec
#Jupyter should create an interactive plot wit qt
%matplotlib qt

#Linear Algebraic, signal processing
import scipy.linalg as scLinAlg
import scipy.signal as sigP

#sympi
import sympy as sypy

#gurobi
#import gurobipy as gp
#from gurobipy import GRB
#import numpy as np

In [2]:
# individual packages
import sg, sa, sp, obq
mtplt.close('all')

sNbins = 2**4
sFs = sNbins
sT = 1 / (sFs)
       
sL = 15
sBSize = 2
sPadLen = sNbins + (sL - 1)
sRecFilterFrequ = 4 

Generate input signal $x_n$

In [3]:
v_n = np.arange(sNbins).reshape(-1, 1)
vxFrequ = (np.arange(0, 4, step=1) * sFs / sNbins).reshape(-1, 1)
vxPhaseInit = np.random.rand(len(vxFrequ), 1) * 2 * np.pi

vx, vTime = sg.signalGen(v_n, vxFrequ, vxPhaseInit, sFs, 'real')
vx = sg.MFnormalize(vx, -1, 1)

# Save to a file
#np.save('signal.npy', vx)
#
vx = np.load('signal.npy')

Generate FiltersCoeff and corresp. Matrices

In [4]:
### Generate ideal matrices ###
vRIdeal = sp.idealBinFilt(sNbins, sg.freq2Bin(sRecFilterFrequ, sNbins, sFs), 'normal')
vRIdealvTest = sp.idealBinFilt(42, sg.freq2Bin(sRecFilterFrequ, 42, sFs), 'normal')
mRIdeal = scLinAlg.toeplitz(vRIdeal)

sAtten = 60 # in dB
sBeta = sigP.kaiser_beta(sAtten)
vW = sigP.firwin(sL, (sRecFilterFrequ) / (sFs/2), window=('kaiser', sBeta))

vLsBand = np.array([0, sRecFilterFrequ, sRecFilterFrequ+1, (sFs/2)]) / (sFs/2)
vDesiredGains = np.array([1, 1, 10**(-6), 10**(-6)])
vWLs = sigP.firls(sL, vLsBand, vDesiredGains)
vNormWLs = vWLs / np.sum(vWLs)

# Replace zeros with eps
#vW[vW == 0] = np.finfo(float).eps

mW = sp.convmtx(vW,len(vx),'colWise')
mWNN = sp.convmtx(vW,len(vx),'colWiseNN')

sPadAdd = int(sPadLen - sNbins)
vxPadded = np.pad(vx,(0,sPadAdd))

vymat = mW @ vx
sCut = int(np.floor((sL-1)/2))


mOnes = np.ones((sNbins,sNbins))
mSigDeltaFilt = np.tril(mOnes)

In [5]:
symW = sypy.s

vWrnd = [round(c,4) for c in vWLs]
vecW = sypy.Matrix(vWrnd)

# Generate LaTeX code
ltx_vW = sypy.latex(vecW, mode='inline')

# Output the LaTeX representation
print(f"w = {ltx_vW}")

w = ('$\\left[\\begin{smallmatrix}-0.0055\\\\-0.0324\\\\0.0274\\\\0.047\\\\-0.0806\\\\-0.0578\\\\0.3091\\\\0.5618\\\\0.3091\\\\-0.0578\\\\-0.0806\\\\0.047\\\\0.0274\\\\-0.0324\\\\-0.0055\\end{smallmatrix}\\right]$',)


Quantize the input signal

In [6]:
# vBSequSingle, ve, ve_hat = obq.iterSequQ(vx,mSigDeltaFilt,0)
# print("Single-Iterative solution found!")
#vBSequBlock, vEBlock = obq.fullOpt(vx,mW[sCut:sCut+len(vx)],vBSequSingle) 
# vW = vNormWLs[6::]
# vBSequBlock, vEBlock = obq.iterBlockQ(vx, vW, sBSize)
#vBSequBlock = vBSequBlock[sPadAdd::]

Frequency Analysis

In [7]:
# vBfft = np.fft.fft(vBSequSingle)
# vBfftMag = 20*sa.safelog10(np.abs(vBfft) / np.max(abs(vBfft))) 

# vBBlockfft = np.fft.fft(vBSequBlock)
# vBBlockfftMag = 20*sa.safelog10(np.abs(vBBlockfft) / np.max(abs(vBBlockfft))) 

# vBReckBlockfft = np.fft.fft(vWLs,sNbins)
# vBReckBlockfftMag = 20*sa.safelog10(np.abs(vBReckBlockfft) / np.max(abs(vBReckBlockfft))) 

# vX = np.fft.fft(vx)
# vXMag = 20*sa.safelog10(np.abs(vX) / np.max(abs(vX)))
# # Frequency bins
# vFreq = np.fft.fftfreq(sNbins, sT)

SNR Calculations

In [8]:
# sVX_MSE, sVX_SNRdb, sVX_PSNRdb = sa.evalN(vx-mRIdeal @ vx, vx)
# sVB_MSE, sVB_SNRdb, sVB_PSNRdb = sa.evalN(mRIdeal @ (vx-vBSequSingle), mRIdeal @ vx)
# sVBBlock_MSE, sVBBlock_SNRdb, sVBBlock_PSNRdb = sa.evalN(mRIdeal @ (vx-vBSequBlock), mRIdeal @ vx)

Plots

In [9]:
# ###### PLOTTING ######
# figOne = mtplt.figure()
# Pltgs = gridspec.GridSpec(3, 2)

# pltDiscTime = figOne.add_subplot(Pltgs[0,:])
# pltDiscTime.plot(vx)
# # pltDiscTime.set_title('Input Signal SNR: {snr} dB'.format(snr = round(sVX_SNRdb,2)))
# pltDiscTime.set_xlabel('Samples $n$', fontsize = 11)
# pltDiscTime.set_ylabel('Amplitude', fontsize = 11)
# pltDiscTime.set_xlim([0,sNbins])
# mtplt.minorticks_on()
# mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

# pltObsOne = figOne.add_subplot(Pltgs[1,0])
# pltObsOne.plot(vWLs) #vFreq[:sNbins // 2], vXMag[:sNbins // 2])
# pltObsOne.set_title('Spectrum Input Signal')
# pltObsOne.set_xlabel('Frequency $(Hz)$', fontsize = 11)
# pltObsOne.set_ylabel('Magnitude $(dB)$', fontsize = 11)
# #pltObsOne.set_xlim([0,sNbins/2])
# #pltObsOne.set_ylim([-60,5])
# mtplt.minorticks_on()
# mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

# pltObsTwo = figOne.add_subplot(Pltgs[1,1])
# pltObsTwo.plot(vBReckBlockfftMag[:sNbins // 2])
# pltObsTwo.set_title('Frequency Response of $W$')
# pltObsTwo.set_xlabel('Frequency $(Hz)$', fontsize = 11)
# pltObsTwo.set_ylabel('Magnitude $(dB)$', fontsize = 11)
# pltObsTwo.set_xlim([0,sNbins/2])
# pltObsTwo.set_ylim([-60,5])
# mtplt.minorticks_on()
# mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

# pltFreqOne = figOne.add_subplot(Pltgs[2,0])
# pltFreqOne.plot(vFreq[:sNbins // 2], vBfftMag[:sNbins // 2])
# pltFreqOne.set_title('Frequency Spectrum One SNR: {snr} dB'.format(snr = round(sVB_SNRdb,2)))
# pltFreqOne.set_xlabel('Frequency $(Hz)$', fontsize = 11)
# pltFreqOne.set_ylabel('Magnitude $(dB)$', fontsize = 11)
# pltFreqOne.set_xlim([0,sNbins/2])
# pltFreqOne.set_ylim([-60,5])
# mtplt.minorticks_on()
# mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

# pltFreqTwo = figOne.add_subplot(Pltgs[2,1])
# pltFreqTwo.plot(vFreq[:sNbins // 2], vBBlockfftMag[:sNbins // 2])
# pltFreqTwo.set_title('Frequency Spectrum Two SNR: {snr} dB'.format(snr = round(sVBBlock_SNRdb,2)))
# pltFreqTwo.set_xlabel('Frequency $(Hz)$', fontsize = 11)
# pltFreqTwo.set_ylabel('Magnitude $(dB)$', fontsize = 11)
# mtplt.minorticks_on()
# pltFreqTwo.set_xlim([0,sNbins/2])
# pltFreqTwo.set_ylim([-60,5])
# mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

# mtplt.tight_layout(pad=-0.25)
# mtplt.show()